<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/data_collection_photos_for_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import of the training data

from google.colab import files
uploaded = files.upload()

# Import of moduls

!pip install wikidata --quiet

import matplotlib.pyplot as plt
import pandas as pd
from wikidata.client import Client
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO

from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Loading the data
train_df = pd.read_csv('[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv', sep='\t')

Saving [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv to [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv


In [2]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]

In [3]:
def get_claims(wikidata_url):
  # Extract the id of the item
    entity_id = extract_entity_id(wikidata_url)

    # Extract information of the item
    client = Client()
    item = client.get(entity_id, load=True)

    # Extract the claims of the item
    claims = item.data.get("claims", {})

    return claims

In [4]:
def get_sitelinks(wikidata_url):
  # Extract the id of the item
    entity_id = extract_entity_id(wikidata_url)

    # Extract information of the item
    client = Client()
    item = client.get(entity_id, load=True)

    # Extract the claims of the item
    sitelinks = item.data.get("sitelinks", {})

    return sitelinks

def get_en_wikipedia_url(wikidata_url):

  sitelinks = get_sitelinks(wikidata_url)

  # Find the english wikidepia page
  enwiki = sitelinks.get("enwiki")

  # Return its URL
  return(enwiki.get("url"))

# Test
wikidata_url = train_df.iloc[0,0]
get_en_wikipedia_url(wikidata_url)

'https://en.wikipedia.org/wiki/Sebasti%C3%A1n_Pi%C3%B1era'

In [19]:
def get_wikipedia_image(wikidata_url):

    # Extract the english wikipedia page if it exists
    try:
      wikipedia_url = get_en_wikipedia_url(wikidata_url)

    except Exception:
      return None

    response = requests.get(wikipedia_url)

    # Make sur the response has no error
    if response.status_code == 200:

        # Extract the infobox of the wikipedia page
        soup = BeautifulSoup(response.text, 'html.parser')
        infobox = soup.find('table', {'class': 'infobox'})

        if infobox:
            # Find the image
            img_tag = infobox.find('img')
            if img_tag:
                # Recreate the URL of the image
                img_url = 'https:' + img_tag['src']
                try:
                  response = requests.get(img_url)
                  try:
                    # Store the image, and return it
                    img = Image.open(BytesIO(response.content))
                    np_array = np.array(img)
                    return np_array
                  except Exception:
                    pass
                except Exception:
                  pass
    # If there is an error somewhere, return None
    return None

# Test
wikidata_url = train_df.iloc[1,0]
image = get_wikipedia_image(wikidata_url)
print(image)

[[[39 37 22]
  [35 33 18]
  [34 32 17]
  ...
  [37 35 23]
  [32 30 18]
  [32 30 18]]

 [[30 28 13]
  [31 29 14]
  [33 31 16]
  ...
  [33 31 19]
  [31 29 17]
  [30 28 16]]

 [[26 24  9]
  [32 30 15]
  [33 31 16]
  ...
  [30 28 16]
  [31 29 17]
  [30 28 16]]

 ...

 [[32 32 22]
  [19 19  9]
  [22 22 14]
  ...
  [30 27 18]
  [23 20 11]
  [23 20 11]]

 [[26 23 14]
  [24 21 12]
  [31 28 21]
  ...
  [28 25 16]
  [25 22 13]
  [25 22 13]]

 [[31 28 19]
  [34 31 22]
  [38 35 28]
  ...
  [31 28 19]
  [32 29 20]
  [32 29 20]]]


As it takes hours to run, when we apply the extraction of the photos on the whole dataset, we slice it, and it runs faster

In [20]:
# Application of the extraction of the photos, for the rows 0:500

# Initialization of the new variable
train_df['image'] = None

# Extraction of the images to the sub dataframe
train_df_subset = train_df[0:500]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[0:500] = train_df_subset

# Visualisation
train_df

<ipython-input-20-beaea87961a1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))


,item,name,description,type,category,subcategory,label,image
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,None
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,"[[[39, 37, 22], [35, 33, 18], [34, 32, 17], [3..."
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,"[[[111, 112, 104], [111, 112, 104], [112, 113,..."
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,"[[[126, 82, 35], [125, 81, 34], [127, 81, 32],..."
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,"[[[107, 89, 77], [107, 89, 77], [104, 86, 74],..."
...,...,...,...,...,...,...,...,...
6246,http://www.wikidata.org/entity/Q321103,Bühl,"quarter of Tübingen, Baden-Württemberg, Germany",entity,geography,neighborhood,cultural exclusive,None
6247,http://www.wikidata.org/entity/Q338167,Tenderloin,area of New York City during the late 19th and...,entity,geography,neighborhood,cultural exclusive,None
6248,http://www.wikidata.org/entity/Q66991,Schinznach-Dorf,former municipality and current district of Sc...,entity,geography,neighborhood,cultural exclusive,None
6249,http://www.wikidata.org/entity/Q66922,Ependes,village and former municipality in Bois-d'Amon...,entity,geography,neighborhood,cultural exclusive,None


In [21]:
# Extraction of the images to the sub dataframe
train_df_subset = train_df[500:1000]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[500:1000] = train_df_subset

# Visualisation
train_df[500:1000]

<ipython-input-21-712185269597>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))


KeyboardInterrupt: 

In [ ]:
# Extraction of the images to the sub dataframe
train_df_subset = train_df[1000:1500]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[1000:1500] = train_df_subset

# Visualisation
train_df[1000:1500]

In [ ]:
# Extraction of the images to the sub dataframe
train_df_subset = train_df[1500:2000]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[1500:2000] = train_df_subset

# Visualisation
train_df[1500:2000]

In [ ]:
# Extraction of the images to the sub dataframe
train_df_subset = train_df[2000:2500]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[2000:2500] = train_df_subset

# Visualisation
train_df[2000:2500]

In [ ]:
# Extraction of the images to the sub dataframe
train_df_subset = train_df[2500:3000]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[2500:3000] = train_df_subset

# Visualisation
train_df[2500:3000]

In [ ]:
# Extraction of the images to the sub dataframe
train_df_subset = train_df[3000:3500]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[3000:3500] = train_df_subset

# Visualisation
train_df[3000:3500]

In [ ]:
# Extraction of the images to the sub dataframe
train_df_subset = train_df[3500:4000]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[3500:4000] = train_df_subset

# Visualisation
train_df[3500:4000]

In [ ]:
# Extraction of the images to the sub dataframe
train_df_subset = train_df[4000:4500]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[4000:4500] = train_df_subset

# Visualisation
train_df[4000:4500]

In [ ]:
# Extraction of the images to the sub dataframe
train_df_subset = train_df[4500:5000]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[4500:5000] = train_df_subset

# Visualisation
train_df[4500:5000]

In [ ]:
# Extraction of the images to the sub dataframe
train_df_subset = train_df[5000:5500]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[5000:5500] = train_df_subset

# Visualisation
train_df[5000:5500]

In [ ]:
# Extraction of the images to the sub dataframe
train_df_subset = train_df[5500:6000]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[5500:6000] = train_df_subset

# Visualisation
train_df[5500:6000]

In [ ]:
# Extraction of the images to the sub dataframe
train_df_subset = train_df[6000:]
train_df_subset['image'] = train_df_subset['item'].apply(lambda x: get_wikipedia_image(x))

# Plug in the training dataset
train_df[6000:] = train_df_subset

# Visualisation
train_df[6000:]

In [ ]:
train_df['image'].count()

In [ ]:
train_df.to_csv('train_df_images.csv', index=False)

In [ ]:
from google.colab import files
files.download('train_df_images.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>